# Folium Project

## Description

### Installation Packages

In [89]:
%pip install pandas sqlalchemy

Note: you may need to restart the kernel to use updated packages.


In [90]:
%pip install pyodbc

In [91]:
%pip install folium

Note: you may need to restart the kernel to use updated packages.


### Code

#### Preparation

In [92]:
import pyodbc
import pandas as pd

# Définir les informations de connexion
# Aller dans SSMIS -> clique droit sur AdventureWorks2022 -> Properties et récupérer le nom du server et de la database
server = 'LAPTOP-SQ0G8B4V\SQLEXPRESS'
database = 'AdventureWorks2022'

# Établir la connexion
conn = pyodbc.connect(
    f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};Windows Authentication"
)


In [93]:
# Exécutez des requêtes SQL et traitez les résultats avec pandas
query = "SELECT V.[Name], A.AddressID, A.AddressLine1, A.AddressLine2, A.City, A.PostalCode, A.SpatialLocation  FROM Purchasing.Vendor V INNER JOIN Person.BusinessEntityAddress BEA ON V.BusinessEntityID = BEA.BusinessEntityID INNER JOIN Person.Address A ON BEA.AddressID = A.AddressID"
data = pd.read_sql(query, conn)


C:\Users\veyss\AppData\Local\Temp\ipykernel_79616\934731447.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(query, conn)


In [94]:
data

,Name,AddressID,AddressLine1,AddressLine2,City,PostalCode,SpatialLocation
0,Signature Cycles,387,50 Kim Court,None,Coronado,92118,b'\xe6\x10\x00\x00\x01\x0c\xee\x1f\x03m\xcfV@@...
1,SUPERSALES INC.,393,9443 Oaxaca,None,Lakewood,90712,b'\xe6\x10\x00\x00\x01\x0c\x92 \xf9\xf0\x14\xe...
2,Lindell,369,9151 Napa C.,None,Lebanon,97355,b'\xe6\x10\x00\x00\x01\x0ciG\x81by>F@iF\xd2\xd...
3,Fitness Association,340,8629 Pepper Place,None,Bellevue,98004,b'\xe6\x10\x00\x00\x01\x0c\xa7\xd7\x1b\xfb@\xd...
4,A. Datum Corporation,346,2596 Big Canyon Road,None,New York,10007,b'\xe6\x10\x00\x00\x01\x0cK\xf4If\xc1UD@\xa3\x...
...,...,...,...,...,...,...,...
99,Bloomington Multisport,314,218 Fall Creek Road,None,West Covina,91791,b'\xe6\x10\x00\x00\x01\x0crY\xabj%\x06A@Bo\xc3...
100,Carlson Specialties,318,2313 B Southampton Rd,None,Missoula,59801,b'\xe6\x10\x00\x00\x01\x0c\xc6\x9b\xda\xe22eG@...
101,"Compete, Inc.",325,9491 Toyon Dr,None,Dallas,75201,b'\xe6\x10\x00\x00\x01\x0cCF\x16\n\xa2d@@c@\xf...
102,Chicago City Saddles,320,3 Gehringer Drive,None,Daly City,94015,b'\xe6\x10\x00\x00\x01\x0c\xf5e\xe7\x8a=\xd1B@...


#### Folium 

In [95]:
import binascii
import json
from IPython.display import display
import folium

In [96]:
# Convertir les données binaires hexadécimales en une chaîne hexadécimale
data['Coordinate'] = data['SpatialLocation'].apply(lambda x: binascii.hexlify(x).decode('utf-8'))
data

,Name,AddressID,AddressLine1,AddressLine2,City,PostalCode,SpatialLocation,Coordinate
0,Signature Cycles,387,50 Kim Court,None,Coronado,92118,b'\xe6\x10\x00\x00\x01\x0c\xee\x1f\x03m\xcfV@@...,e6100000010cee1f036dcf564040d712f55e2c435dc0
1,SUPERSALES INC.,393,9443 Oaxaca,None,Lakewood,90712,b'\xe6\x10\x00\x00\x01\x0c\x92 \xf9\xf0\x14\xe...,e6100000010c9220f9f014e640405ad9de779b885dc0
2,Lindell,369,9151 Napa C.,None,Lebanon,97355,b'\xe6\x10\x00\x00\x01\x0ciG\x81by>F@iF\xd2\xd...,e6100000010c69478162793e46406946d2d206b35ec0
3,Fitness Association,340,8629 Pepper Place,None,Bellevue,98004,b'\xe6\x10\x00\x00\x01\x0c\xa7\xd7\x1b\xfb@\xd...,e6100000010ca7d71bfb40d44740e6714174ca985ec0
4,A. Datum Corporation,346,2596 Big Canyon Road,None,New York,10007,b'\xe6\x10\x00\x00\x01\x0cK\xf4If\xc1UD@\xa3\x...,e6100000010c4bf44966c1554440a3fd3323498652c0
...,...,...,...,...,...,...,...,...
99,Bloomington Multisport,314,218 Fall Creek Road,None,West Covina,91791,b'\xe6\x10\x00\x00\x01\x0crY\xabj%\x06A@Bo\xc3...,e6100000010c7259ab6a25064140426fc3af78845dc0
100,Carlson Specialties,318,2313 B Southampton Rd,None,Missoula,59801,b'\xe6\x10\x00\x00\x01\x0c\xc6\x9b\xda\xe22eG@...,e6100000010cc69bdae232654740c25736db767e5cc0
101,"Compete, Inc.",325,9491 Toyon Dr,None,Dallas,75201,b'\xe6\x10\x00\x00\x01\x0cCF\x16\n\xa2d@@c@\xf...,e6100000010c4346160aa26440406340f0e64f3b58c0
102,Chicago City Saddles,320,3 Gehringer Drive,None,Daly City,94015,b'\xe6\x10\x00\x00\x01\x0c\xf5e\xe7\x8a=\xd1B@...,e6100000010cf565e78a3dd14240d29aa7b63ea55ec0


In [97]:
# Créez deux listes vides pour stocker les valeurs de latitude et de longitude
latitude_values = []
longitude_values = []

# Extrayez les coordonnées à partir des chaînes hexadécimales et ajoutez-les aux listes
for hex_string in data['Coordinate']:
    binary_data = bytes.fromhex(hex_string)
    latitude = binary_data[0:8]
    longitude = binary_data[8:16]
    latitude_value = int.from_bytes(latitude, byteorder='little', signed=True) /1e17
    longitude_value = int.from_bytes(longitude, byteorder='little', signed=True) / 1e17
    latitude_values.append(latitude_value)
    longitude_values.append(longitude_value)

# Ajoutez les listes de valeurs de latitude et de longitude au DataFrame
data['Latitude'] = latitude_values
data['Longitude'] = longitude_values

# Affichez le DataFrame mis à jour
print(data[["Latitude", "Longitude"]])


      Latitude  Longitude
0    23.007897  13.576245
1    23.469516 -27.848428
2    51.456572  50.736571
3   -29.073418  82.073180
4    -8.435673  -1.702217
..         ...        ...
99   64.452272  80.170409
100 -72.220718  63.236875
101  50.629036  46.396227
102  73.467916 -72.906942
103  21.127644 -66.759528

[104 rows x 2 columns]


In [98]:
data = data.drop(columns= ["SpatialLocation","Coordinate"])
data

,Name,AddressID,AddressLine1,AddressLine2,City,PostalCode,Latitude,Longitude
0,Signature Cycles,387,50 Kim Court,None,Coronado,92118,23.007897,13.576245
1,SUPERSALES INC.,393,9443 Oaxaca,None,Lakewood,90712,23.469516,-27.848428
2,Lindell,369,9151 Napa C.,None,Lebanon,97355,51.456572,50.736571
3,Fitness Association,340,8629 Pepper Place,None,Bellevue,98004,-29.073418,82.073180
4,A. Datum Corporation,346,2596 Big Canyon Road,None,New York,10007,-8.435673,-1.702217
...,...,...,...,...,...,...,...,...
99,Bloomington Multisport,314,218 Fall Creek Road,None,West Covina,91791,64.452272,80.170409
100,Carlson Specialties,318,2313 B Southampton Rd,None,Missoula,59801,-72.220718,63.236875
101,"Compete, Inc.",325,9491 Toyon Dr,None,Dallas,75201,50.629036,46.396227
102,Chicago City Saddles,320,3 Gehringer Drive,None,Daly City,94015,73.467916,-72.906942


In [99]:
# Créez une carte Folium comme décrit précédemment
m = folium.Map(location = [40, -95], zoom_start = 4)
display(m)

In [100]:
# Ajoutez des marqueurs à la carte
for index, row in data.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row[['Name','City']],
        icon=folium.Icon(color='red'),

    ).add_to(m)

# Affichez la carte dans le notebook
m

#### Fermeture de la connection avec AdventureWorks2022

In [101]:
# Fermez la connexion
conn.close()